# 크롤링한 뉴스 모델 성능 높이기

## Mecab으로 토큰화하기

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import os
from konlpy.tag import Mecab
tokenizer = Mecab()

In [42]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
df = pd.read_table(csv_path, sep=',')
df.head()
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']
df.drop_duplicates(subset=['news'], inplace=True)
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    903
1     사회   1668
2  생활/문화   1423


In [43]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [44]:
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [45]:
import time
start = time.time()
text_data = preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포
time : 4.940153121948242


In [46]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [47]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 2995
테스트용 뉴스 기사의 개수 :  999
훈련용 레이블의 개수 :  2995
테스트용 레이블의 개수 :  999


In [48]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [49]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [50]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.81      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.80      0.81       999
weighted avg       0.82      0.82      0.82       999



### Mecab의 장점은 가장 빠른 시간을 기록했다는 것에 있다. 다른 형태소 분석기는 1분을 넘긴 것에 비해 Mecab은 4초밖에 걸리지 않았다. 그렇지만 정확도에서 82%를 기록하고 가중평균 f1-score도 82%를 유지한다. 종합 결과를 볼때 Mecab이 가장 적합한 형태소분석기라고 생각한다.

---

## Hannanum으로 토큰화하기

In [10]:
from konlpy import init_jvm 
from konlpy.tag import Hannanum 
init_jvm() 
hannanum = Hannanum()

In [11]:
def preprocessing_hannanum(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = hannanum.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [12]:
import time
start = time.time()
text_data2 = preprocessing_hannanum(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포
time : 154.8171546459198


In [13]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(text_data2, df['code'], random_state = 0)

In [14]:
count_vect = CountVectorizer()
X_train_2_counts = count_vect.fit_transform(X_train_2)

tfidf_transformer = TfidfTransformer()
X_train_2_tfidf = tfidf_transformer.fit_transform(X_train_2_counts)

clf = MultinomialNB().fit(X_train_2_tfidf, y_train_2)

In [15]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [16]:
y_pred_2 = clf.predict(tfidf_vectorizer(X_test_2))
print(metrics.classification_report(y_test_2, y_pred_2))

              precision    recall  f1-score   support

       IT/과학       0.90      0.64      0.75       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.79      0.79      0.79       341

    accuracy                           0.81       999
   macro avg       0.83      0.78      0.80       999
weighted avg       0.82      0.81      0.81       999



### Hannanum같은 경우 154초라는 시간이 결렸지만 정확도는 81%, 가중평균 f1-score로 오히려 Mecab에 밀린다. 실험 결과에서 Hannanum은 가장 최적화된 형태소분석기라고 보기 어렵다.


---

## kkma으로 토큰화하기

In [17]:
from konlpy.tag import Kkma
kkma = Kkma()

In [18]:
def preprocessing_kkma(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = kkma.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [20]:
import time
start = time.time()
text_data3 = preprocessing_kkma(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포
time : 1098.8279449939728


In [21]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(text_data3, df['code'], random_state = 0)

In [22]:
count_vect = CountVectorizer()
X_train_3_counts = count_vect.fit_transform(X_train_3)

tfidf_transformer = TfidfTransformer()
X_train_3_tfidf = tfidf_transformer.fit_transform(X_train_3_counts)

clf = MultinomialNB().fit(X_train_3_tfidf, y_train_3)

In [23]:
y_pred_3 = clf.predict(tfidf_vectorizer(X_test_3))
print(metrics.classification_report(y_test_3, y_pred_3))

              precision    recall  f1-score   support

       IT/과학       0.90      0.75      0.82       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



### Kkma 형태소 분석기의 성능은 정확도 82%, 가중평균 f1-score 82%라서 Mecab과 동등한 성능을 보여주지만 1098초 (18.3분)이라는 어마어마한 시간이 걸린다. 그렇기에 효율성이라는 측면에서 Kkma형태소 분석기는 좋지 않다고 생각한다.

---

## Komoran으로 토큰화하기

In [25]:
from konlpy.tag import Komoran
komoran = Komoran()

In [26]:
def preprocessing_komoran(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = komoran.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [27]:
import time
start = time.time()
text_data4 = preprocessing_komoran(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포
time : 62.95089364051819


In [28]:
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(text_data4, df['code'], random_state = 0)

In [29]:
count_vect = CountVectorizer()
X_train_4_counts = count_vect.fit_transform(X_train_4)

tfidf_transformer = TfidfTransformer()
X_train_4_tfidf = tfidf_transformer.fit_transform(X_train_4_counts)

clf = MultinomialNB().fit(X_train_4_tfidf, y_train_4)

In [30]:
y_pred_4 = clf.predict(tfidf_vectorizer(X_test_4))
print(metrics.classification_report(y_test_4, y_pred_4))

              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



### Komoran 형태소 분석기는 정확도는 82%, 가중 f1-score도 82%로 Mecab이랑 동등한 결과를 나타냈지만 시간이 62초가 걸려 상대적으로 Mecab이 더 좋은 형태소 분석기라 볼 수 있다.

---

## Twitter 형태소 분석기

In [33]:
from konlpy.tag import Twitter
twitter = Twitter()

/home/aiffel0038/anaconda3/envs/aiffel/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [34]:
def preprocessing_twitter(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = twitter.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [37]:
import time
start = time.time()
text_data5 = preprocessing_twitter(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포
time : 141.47270035743713


In [38]:
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(text_data5, df['code'], random_state = 0)

In [39]:
count_vect = CountVectorizer()
X_train_5_counts = count_vect.fit_transform(X_train_5)

tfidf_transformer = TfidfTransformer()
X_train_5_tfidf = tfidf_transformer.fit_transform(X_train_5_counts)

clf = MultinomialNB().fit(X_train_5_tfidf, y_train_5)

In [40]:
y_pred_5 = clf.predict(tfidf_vectorizer(X_test_5))
print(metrics.classification_report(y_test_5, y_pred_5))

              precision    recall  f1-score   support

       IT/과학       0.91      0.70      0.79       236
          사회       0.78      0.93      0.85       422
       생활/문화       0.83      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.80      0.81       999
weighted avg       0.83      0.82      0.82       999



### Twitter 분석기는 정확도나 가중평균은 Mecab과 동일하지만 시간에서 141초가 걸려 Mecab보다 좋은 형태소 분석기라고 보기 어렵다.

---

## 불용어 추가하기

In [236]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [237]:
stopwords.extend(['카카오톡', '재', '배포', '도', '아', '씨', '로', '고', '를',
                  '로', '것', '으로', '다고', '라고', '으로', '와', '로', '위해', '기', '놓', '잘', '좀', '개', '어떻', '어떤', '가지'])

In [238]:
stopwords

['에',
 '는',
 '은',
 '을',
 '했',
 '에게',
 '있',
 '이',
 '의',
 '하',
 '한',
 '다',
 '과',
 '때문',
 '할',
 '수',
 '무단',
 '따른',
 '및',
 '금지',
 '전재',
 '경향신문',
 '기자',
 '는데',
 '가',
 '등',
 '들',
 '파이낸셜',
 '저작',
 '등',
 '뉴스',
 '카카오톡',
 '재',
 '배포',
 '도',
 '아',
 '씨',
 '로',
 '고',
 '를',
 '로',
 '것',
 '으로',
 '다고',
 '라고',
 '으로',
 '와',
 '로',
 '위해',
 '기',
 '놓',
 '잘',
 '좀',
 '개',
 '어떻',
 '어떤',
 '가지']

In [239]:
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [240]:
import time
start = time.time()
text_data= preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 어 시민 국비 지원금 만 지급 며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 긴급 재난 지원금 추진 태 스 크 포스 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자
time : 5.373556137084961


In [241]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [242]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [243]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [244]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.80      0.82       999
weighted avg       0.82      0.82      0.82       999



### 불용어를 20개 정도 추가했을때는 큰 드라마틱한 변화는 없고 단순 f1-score 평균이 1% 올랐다.

In [245]:
words_txt_path = os.getenv("HOME") + "/aiffel/news_crawler/한국어불용어100.txt"
stopwords_100 = pd.read_table(words_txt_path)
stopwords_100['이'] = new_stopwords

In [246]:
new_stopwords = list(new_stopwords)

In [247]:
stopwords.extend(new_stopwords)
stopwords.extend('한다')

In [248]:
words2_txt_path = os.getenv("HOME") + "/aiffel/news_crawler/stopwords-ko.txt"
stopwords_2 = pd.read_table(words2_txt_path)

In [249]:
stopwords.extend(stopwords_2)

In [251]:
import time
start = time.time()
text_data= preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 시 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 시민 긴급 재난 지원금 인 이상 가구 기준 만 게 며 인 가구 만 인 가구 만 인 가구 만 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 이유 국비 지방비 부담 비율 다파 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 지급 어 시민 국비 지원금 만 지급 며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만 경기도 재난 기본소득 만 정부 긴급 재난 지원금 만 총 만 게 된다 취약 계층 아닌 시민 월일 부터 소지 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 시 김정기 부시장 단장 긴급 재난 지원금 추진 태 스 포스 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자
time : 6.116743803024292


In [252]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.90      0.75      0.81       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.82      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.81      0.82       999
weighted avg       0.82      0.82      0.82       999



### 불용어를 많이 추가해 정확도와 f1 score를 늘리려고 헀지만 여전히 변화가 미흡했다.

---

## 데이터셋 추가하기

In [253]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data2.csv"
df2 = pd.read_table(csv_path, sep=',')
df2.head()
df2['news'] = df2['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df2['news']
df2.drop_duplicates(subset=['news'], inplace=True)
idx = df2[df2['code'] == '경제'].index
df3 =df2.drop(idx)
print(df3.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    235
1     사회    554
2  생활/문화    446


In [254]:
new_df= pd.concat([df, df3])
new_df

,news,code
0,파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...,사회
1,동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...,사회
4,서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...,사회
...,...,...
3194,레고켐바이오사이언스대표 김용주사진가 글로벌 기술수출에 또다시 성공했다 년 중국 포순...,IT/과학
3195,제대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 일 오후 시부...,IT/과학
3196,아마존 연합뉴스 자료사진 아마존 연합뉴스 자료사진직원들 코로나 확산 방지 조치 충분...,IT/과학
3197,제대 국회의원선거가 임박한 가운데 투표 및 개표 방송을 준비하는 기업들의 움직임이 ...,IT/과학


In [255]:
import time
start = time.time()
text_data = preprocessing(new_df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 시 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 시민 긴급 재난 지원금 인 이상 가구 기준 만 게 며 인 가구 만 인 가구 만 인 가구 만 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 이유 국비 지방비 부담 비율 다파 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 지급 어 시민 국비 지원금 만 지급 며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만 경기도 재난 기본소득 만 정부 긴급 재난 지원금 만 총 만 게 된다 취약 계층 아닌 시민 월일 부터 소지 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 시 김정기 부시장 단장 긴급 재난 지원금 추진 태 스 포스 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자
time : 8.32047724723816


In [256]:
X_train, X_test, y_train, y_test = train_test_split(text_data, new_df['code'], random_state = 0)

In [257]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [258]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [259]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.90      0.72      0.80       285
          사회       0.79      0.93      0.85       547
       생활/문화       0.84      0.77      0.80       476

    accuracy                           0.82      1308
   macro avg       0.84      0.81      0.82      1308
weighted avg       0.83      0.82      0.82      1308



### 노션에 올라온 데이터를 추가했지만 그다지 변한 것이 없었다.. 하지만 이때 떠오른 건 가위바위보 게임이었다. 가위바위보 게임에서 내가 얻은 결론은 일단 데이터를 많이 확보해서 정확도를 늘려보자였다. 그래서 나는 크롤링을 통해 데이터셋의 크기를 늘렸다.

In [211]:

import requests
import pandas as pd
from bs4 import BeautifulSoup


def make_urllist(page_num, code, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
    news = requests.get(url)

    
    soup = BeautifulSoup(news.content, 'html.parser')

    
    news_list = soup.select('.newsflash_body .type06_headline li dl')
   
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [223]:
from newspaper import Article
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}
.
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)


  df = pd.DataFrame({'news': text_list})


  df['code'] = idx2word[str(code)]
  return df

In [224]:
def make_total_data(page_num, code_list, date):
  df = None

  for code in code_list:
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

In [225]:
code_list = [102, 103, 105]

In [226]:
df_4 = make_total_data(10, code_list, 20200504)

102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


In [227]:
import os

csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data_3.csv"
df_4.to_csv(csv_path, index=False)

if os.path.exists(csv_path):
  print('{} File Saved!'.format(csv_path))

/home/aiffel0038/aiffel/news_crawler/news_data_3.csv File Saved!


In [260]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data_3.csv"
df4 = pd.read_table(csv_path, sep=',')
df4.head()
df4['news'] = df4['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df4['news']
df4.drop_duplicates(subset=['news'], inplace=True)
print(df4.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    198
1     사회    191
2  생활/문화    189


In [261]:
new_df_2 = pd.concat([new_df, df4])

In [262]:
import time
start = time.time()
text_data = preprocessing(new_df_2['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 시 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 시민 긴급 재난 지원금 인 이상 가구 기준 만 게 며 인 가구 만 인 가구 만 인 가구 만 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 이유 국비 지방비 부담 비율 다파 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 지급 어 시민 국비 지원금 만 지급 며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만 경기도 재난 기본소득 만 정부 긴급 재난 지원금 만 총 만 게 된다 취약 계층 아닌 시민 월일 부터 소지 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 시 김정기 부시장 단장 긴급 재난 지원금 추진 태 스 포스 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자
time : 8.494166135787964


In [263]:
X_train, X_test, y_train, y_test = train_test_split(text_data, new_df_2['code'], random_state = 0)

In [264]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [265]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.88      0.78      0.83       331
          사회       0.84      0.92      0.87       595
       생활/문화       0.84      0.81      0.82       526

    accuracy                           0.85      1452
   macro avg       0.85      0.84      0.84      1452
weighted avg       0.85      0.85      0.85      1452



### 데이터셋을 늘리니 획기적으로 정확도가 2%가 올랐다. 역시.. 데이터셋 크기를 늘려주는 것이 매우 중요하다는 것을 다시금 깨달았다.

---

## 부록: 데이터셋을 늘리지 않고 단순히 모델을 바꾼다면 어떻게 변화할까?

In [266]:
import time
start = time.time()
text_data= preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 시 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 시민 긴급 재난 지원금 인 이상 가구 기준 만 게 며 인 가구 만 인 가구 만 인 가구 만 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 이유 국비 지방비 부담 비율 다파 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 지급 어 시민 국비 지원금 만 지급 며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만 경기도 재난 기본소득 만 정부 긴급 재난 지원금 만 총 만 게 된다 취약 계층 아닌 시민 월일 부터 소지 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 시 김정기 부시장 단장 긴급 재난 지원금 추진 태 스 포스 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자
time : 7.241555213928223


In [269]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [270]:
from sklearn.ensemble import RandomForestClassifier
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = RandomForestClassifier().fit(X_train_tfidf, y_train)

In [271]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.86      0.83      0.84       236
          사회       0.85      0.89      0.87       422
       생활/문화       0.83      0.79      0.81       341

    accuracy                           0.84       999
   macro avg       0.84      0.84      0.84       999
weighted avg       0.84      0.84      0.84       999



In [272]:
from sklearn.linear_model import SGDClassifier
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_tran데이터를sform(X_train_counts)

clf = SGDClassifier().fit(X_train_tfidf, y_train)

In [273]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.88      0.86      0.87       236
          사회       0.90      0.88      0.89       422
       생활/문화       0.83      0.86      0.84       341

    accuracy                           0.87       999
   macro avg       0.87      0.87      0.87       999
weighted avg       0.87      0.87      0.87       999



### 모델을 바꾸게 되는 것 또한 데이터셋을 늘려주는 효과를 나타났다.